# **AI TECH INSTITUTE** · *Intermediate AI & Data Science*
### Week 8 - Notebook 01: Linear Models Complete Guide
**Instructor:** Amir Charkhi | **Goal:** Master all linear modeling techniques

> From simple regression to advanced regularization

## 📚 What You'll Learn Today

**Linear Models Covered:**
1. ✅ Simple Linear Regression (1 feature)
2. ✅ Multiple Linear Regression (many features)
3. ✅ Polynomial Regression (curved relationships)
4. ✅ Logistic Regression (classification)
5. ✅ Ridge Regression (L2 regularization)
6. ✅ Lasso Regression (L1 regularization)
7. ✅ ElasticNet (L1 + L2 combined)

**Why Linear Models?**
- Fast to train
- Easy to interpret
- Work well with small datasets
- Great baseline models
- Often used in production

**Time**: 90 minutes | **Prerequisites**: Week 7

---

In [ ]:
# Essential imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

print("📊 WEEK 8: LINEAR MODELS")
print("✅ Setup complete!")

In [ ]:
# ML imports
from sklearn.linear_model import (
    LinearRegression,
    LogisticRegression,
    Ridge,
    Lasso,
    ElasticNet
)
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (
    mean_squared_error,
    r2_score,
    accuracy_score,
    classification_report
)

print("✅ ML libraries loaded!")

---

## Part 1: Simple Linear Regression

**The most basic form:** Predict Y from one X

**Equation:** `y = mx + b`
- m = slope (how much Y changes when X increases by 1)
- b = intercept (Y when X=0)

**Real Example:** Predict house price from square footage

### 1.1: Create Sample Data

In [ ]:
# Generate synthetic house data
np.random.seed(42)
n_houses = 100

# Square footage (1000-3000 sq ft)
sqft = np.random.uniform(1000, 3000, n_houses)

# Price: $100 per sq ft + some randomness
price = 100 * sqft + np.random.normal(0, 20000, n_houses)

print(f"Created {n_houses} house samples")
print(f"Square footage range: {sqft.min():.0f} - {sqft.max():.0f}")
print(f"Price range: ${price.min():,.0f} - ${price.max():,.0f}")

### 1.2: Visualize the Relationship

In [ ]:
# Scatter plot of data
plt.figure(figsize=(10, 6))
plt.scatter(sqft, price, alpha=0.6, color='steelblue', s=50)
plt.xlabel('Square Footage', fontsize=12)
plt.ylabel('Price ($)', fontsize=12)
plt.title('House Price vs Square Footage', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 Notice the linear relationship!")

### 1.3: Prepare Data

In [ ]:
# Reshape for sklearn (needs 2D array)
X = sqft.reshape(-1, 1)
y = price

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print("\n✅ Data ready for modeling!")

### 1.4: Train Linear Regression Model

In [ ]:
# Create and fit the model
model = LinearRegression()
model.fit(X, y)

print("✅ Model trained!")
print(f"\nLearned equation: Price = {model.coef_[0]:.2f} × sqft + {model.intercept_:,.0f}")
print(f"\nInterpretation:")
print(f"  - Each extra sq ft adds ${model.coef_[0]:.2f} to price")
print(f"  - Base price (0 sq ft) would be ${model.intercept_:,.0f}")

### 1.5: Make Predictions

In [ ]:
# Predict prices
y_pred = model.predict(X)

# Show some examples
print("Sample Predictions:")
for i in range(5):
    print(f"  {sqft[i]:.0f} sq ft → Predicted: ${y_pred[i]:,.0f}, Actual: ${y[i]:,.0f}")

### 1.6: Visualize Predictions

In [ ]:
# Plot data and regression line
plt.figure(figsize=(10, 6))
plt.scatter(sqft, price, alpha=0.6, color='steelblue', s=50, label='Actual')
plt.plot(sqft, y_pred, color='red', linewidth=2, label='Predicted (Best Fit Line)')
plt.xlabel('Square Footage', fontsize=12)
plt.ylabel('Price ($)', fontsize=12)
plt.title('Linear Regression: Price vs Square Footage', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 1.7: Evaluate Performance

In [ ]:
# Calculate metrics
mse = mean_squared_error(y, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y, y_pred)

print("Model Performance:")
print(f"  RMSE: ${rmse:,.0f}")
print(f"  R² Score: {r2:.4f}")
print(f"\n💡 Interpretation:")
print(f"  - Predictions are off by ${rmse:,.0f} on average")
print(f"  - Model explains {r2*100:.1f}% of price variance")

### 1.8: Make New Predictions

In [ ]:
# Predict for new houses
new_houses = np.array([[1500], [2000], [2500]])
predictions = model.predict(new_houses)

print("Predictions for new houses:")
for sqft, price in zip(new_houses.flatten(), predictions):
    print(f"  {sqft:.0f} sq ft → ${price:,.0f}")

---

## Part 2: Multiple Linear Regression

**Real world:** Multiple features affect the outcome!

**Equation:** `y = b₀ + b₁x₁ + b₂x₂ + ... + bₙxₙ`

**Example:** House price from sqft, bedrooms, bathrooms, age

### 2.1: Create Multi-Feature Dataset

In [ ]:
# Generate realistic house data
np.random.seed(42)
n = 200

data = {
    'sqft': np.random.uniform(1000, 3500, n),
    'bedrooms': np.random.randint(2, 6, n),
    'bathrooms': np.random.uniform(1, 4, n),
    'age': np.random.randint(0, 50, n),
    'garage': np.random.randint(0, 3, n)
}

df = pd.DataFrame(data)
print("Dataset created with multiple features:")
print(df.head())

### 2.2: Create Target Variable

In [ ]:
# Create price with realistic weights
df['price'] = (
    100 * df['sqft'] +              # $100 per sqft
    15000 * df['bedrooms'] +        # $15k per bedroom
    20000 * df['bathrooms'] +       # $20k per bathroom
    -2000 * df['age'] +             # -$2k per year old
    10000 * df['garage'] +          # $10k per garage spot
    np.random.normal(0, 30000, n)   # Random noise
)

print("Price statistics:")
print(df['price'].describe())

### 2.3: Explore Correlations

In [ ]:
# Correlation with price
correlations = df.corr()['price'].sort_values(ascending=False)
print("Feature correlations with price:")
print(correlations)

print("\n💡 Insights:")
print("  - sqft has strongest correlation (largest impact)")
  print("  - age has negative correlation (older = cheaper)")

### 2.4: Visualize Feature Relationships

In [ ]:
# Create correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', center=0, 
            fmt='.2f', square=True, linewidths=1)
plt.title('Feature Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 2.5: Prepare Data for Modeling

In [ ]:
# Separate features and target
X = df[['sqft', 'bedrooms', 'bathrooms', 'age', 'garage']]
y = df['price']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nUsing {X.shape[1]} features to predict price")

### 2.6: Train/Test Split

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")

### 2.7: Train Multiple Linear Regression

In [ ]:
# Train model
mlr = LinearRegression()
mlr.fit(X_train, y_train)

print("✅ Multiple Linear Regression trained!")
print(f"\nIntercept: ${mlr.intercept_:,.0f}")

### 2.8: Interpret Coefficients

In [ ]:
# Display coefficients
coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': mlr.coef_
}).sort_values('Coefficient', ascending=False)

print("\nFeature Coefficients:")
print(coef_df.to_string(index=False))

print("\n💡 Interpretation:")
for feat, coef in zip(coef_df['Feature'], coef_df['Coefficient']):
    if coef > 0:
        print(f"  - {feat}: +${coef:,.0f} per unit increase")
    else:
        print(f"  - {feat}: ${coef:,.0f} per unit increase (decreases price!)")

### 2.9: Make Predictions

In [ ]:
# Predict on test set
y_pred = mlr.predict(X_test)

# Show examples
comparison = pd.DataFrame({
    'Actual': y_test[:5].values,
    'Predicted': y_pred[:5],
    'Error': y_test[:5].values - y_pred[:5]
})

print("Sample Predictions:")
print(comparison.to_string(index=False))

### 2.10: Evaluate Model

In [ ]:
# Calculate metrics
train_r2 = mlr.score(X_train, y_train)
test_r2 = mlr.score(X_test, y_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("Model Performance:")
print(f"  Training R²: {train_r2:.4f}")
print(f"  Test R²: {test_r2:.4f}")
print(f"  RMSE: ${rmse:,.0f}")

if train_r2 - test_r2 < 0.05:
    print("\n✅ No overfitting detected!")
else:
    print("\n⚠️ Model might be overfitting")

### 2.11: Visualize Predictions

In [ ]:
# Plot actual vs predicted
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.6, s=50)
plt.plot([y_test.min(), y_test.max()], 
         [y_test.min(), y_test.max()], 
         'r--', linewidth=2, label='Perfect Predictions')
plt.xlabel('Actual Price ($)', fontsize=12)
plt.ylabel('Predicted Price ($)', fontsize=12)
plt.title('Actual vs Predicted House Prices', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 Points close to red line = good predictions!")

---

## Part 3: Polynomial Regression

**Problem:** What if the relationship isn't linear?

**Solution:** Add polynomial features (x², x³, etc.)

**Example:** Car fuel efficiency vs speed (not linear!)

### 3.1: Generate Non-Linear Data

In [ ]:
# Car speed vs fuel efficiency (has curved relationship)
np.random.seed(42)
speed = np.linspace(20, 100, 100)

# Efficiency peaks around 55 mph, then decreases
efficiency = 10 + 0.8*speed - 0.008*speed**2 + np.random.normal(0, 2, 100)

print(f"Generated {len(speed)} speed-efficiency pairs")
print(f"Speed range: {speed.min():.0f} - {speed.max():.0f} mph")

### 3.2: Visualize Non-Linear Relationship

In [ ]:
# Plot the curved relationship
plt.figure(figsize=(10, 6))
plt.scatter(speed, efficiency, alpha=0.6, s=50, color='green')
plt.xlabel('Speed (mph)', fontsize=12)
plt.ylabel('Fuel Efficiency (mpg)', fontsize=12)
plt.title('Fuel Efficiency vs Speed (Non-Linear)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 Notice the curve! Linear regression won't work well here.")

### 3.3: Try Linear Regression First (will fail!)

In [ ]:
# Prepare data
X_speed = speed.reshape(-1, 1)
y_eff = efficiency

# Train linear model
linear_model = LinearRegression()
linear_model.fit(X_speed, y_eff)
linear_pred = linear_model.predict(X_speed)

# Evaluate
linear_r2 = r2_score(y_eff, linear_pred)
print(f"Linear Model R²: {linear_r2:.4f}")
print("⚠️ Not very good - can't capture the curve!")

### 3.4: Create Polynomial Features

In [ ]:
# Create polynomial features (degree 2)
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X_speed)

print(f"Original features: {X_speed.shape}")
print(f"Polynomial features: {X_poly.shape}")
print(f"\nNew features: [1, speed, speed²]")

### 3.5: Train Polynomial Model

In [ ]:
# Train on polynomial features
poly_model = LinearRegression()
poly_model.fit(X_poly, y_eff)
poly_pred = poly_model.predict(X_poly)

# Evaluate
poly_r2 = r2_score(y_eff, poly_pred)
print(f"Polynomial Model R²: {poly_r2:.4f}")
print("✅ Much better! Can capture the curve!")

### 3.6: Compare Linear vs Polynomial

In [ ]:
# Plot both models
plt.figure(figsize=(12, 6))

plt.scatter(speed, efficiency, alpha=0.6, s=50, color='black', label='Actual Data')
plt.plot(speed, linear_pred, 'r-', linewidth=2, label=f'Linear (R²={linear_r2:.3f})')
plt.plot(speed, poly_pred, 'g-', linewidth=2, label=f'Polynomial (R²={poly_r2:.3f})')

plt.xlabel('Speed (mph)', fontsize=12)
plt.ylabel('Fuel Efficiency (mpg)', fontsize=12)
plt.title('Linear vs Polynomial Regression', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 Green curve fits the data much better!")

---

## Part 4: Logistic Regression (Classification)

**Different task:** Predict categories, not numbers!

**Output:** Probability between 0 and 1

**Example:** Will customer buy? (Yes/No)

### 4.1: Create Classification Dataset

In [ ]:
# E-commerce: Will customer buy based on time on site and page views?
from sklearn.datasets import make_classification

X, y = make_classification(
    n_samples=500,
    n_features=2,
    n_informative=2,
    n_redundant=0,
    random_state=42
)

# Create DataFrame
df_class = pd.DataFrame(X, columns=['time_on_site', 'page_views'])
df_class['purchased'] = y

print(f"Dataset: {len(df_class)} customers")
print(f"\nPurchase rate: {y.mean():.1%}")

### 4.2: Visualize Classification Problem

In [ ]:
# Plot by class
plt.figure(figsize=(10, 6))
colors = ['red', 'green']
labels = ['Did Not Purchase', 'Purchased']

for i in [0, 1]:
    mask = df_class['purchased'] == i
    plt.scatter(df_class[mask]['time_on_site'], 
                df_class[mask]['page_views'],
                c=colors[i], label=labels[i], alpha=0.6, s=50)

plt.xlabel('Time on Site', fontsize=12)
plt.ylabel('Page Views', fontsize=12)
plt.title('Customer Purchase Behavior', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 4.3: Train/Test Split

In [ ]:
# Prepare data
X = df_class[['time_on_site', 'page_views']]
y = df_class['purchased']

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training: {len(X_train)} samples")
print(f"Testing: {len(X_test)} samples")

### 4.4: Train Logistic Regression

In [ ]:
# Train model
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

print("✅ Logistic Regression trained!")
print(f"\nModel coefficients:")
for feature, coef in zip(X.columns, log_reg.coef_[0]):
    print(f"  {feature}: {coef:.4f}")

### 4.5: Make Predictions

In [ ]:
# Predict classes
y_pred = log_reg.predict(X_test)

# Get probabilities
y_pred_proba = log_reg.predict_proba(X_test)

# Show examples
print("Sample Predictions:")
for i in range(5):
    print(f"  Predicted: {y_pred[i]}, Probability: {y_pred_proba[i][1]:.2%}, Actual: {y_test.iloc[i]}")

### 4.6: Evaluate Classification

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Detailed report
print("Classification Performance:")
print(f"\nAccuracy: {accuracy:.1%}")
print("\n" + classification_report(y_test, y_pred, 
                                   target_names=['No Purchase', 'Purchase']))

---

*[Continuing in next part with Regularization...]*